This script 
1. tranfer the wav file into mfcc and saves it as .npy file. 
2. concate the single npy files

In [2]:
import numpy as np
import glob
import sklearn
import librosa
import os

In [3]:
data_path = r'.\data_collection'

In [4]:
def wav_to_mfcc(wav, okNok, curPre):
    """
    Transfer each .wav audio files to mel and save in .npy file seperately to prevent out of memory
    
    Arguments: 
    wav -- .wav audio files
    okNok -- ok: minimal difference (pause, speed, etc); nok: regressions
    curPre -- current: current generated wav; previous: previous baseline
    
    Return:
    XXX_mel.npy saved 
    
    """
    x, fs = librosa.load(wav)
    # display waveplot
    # librosa.display.waveplot(x, fs)
    mel = librosa.feature.mfcc(x, fs, n_mfcc=52)
    # normalize the mel 
    mel = sklearn.preprocessing.scale(mel, axis=1)
    # padding
    mel_padded = librosa.util.fix_length(mel, 600)
    # print the shape the mel
    #print mel_padded.shape
    # display the preprocessed mel
    #librosa.display.specshow(mel_padded)
    
    base_name = os.path.basename(wav).split('.')[0]
    #print(base_name)
    mel_dir = os.path.join(data_path, okNok, curPre)
    if not os.path.exists(mel_dir):
        os.makedirs(mel_dir) 
    mel_name = os.path.join(data_path, okNok, curPre, base_name + '_mel.npy')

    np.save(mel_name,mel_padded)
    return mel_padded
    
def get_wavs(cwd, okNok, curPre):
    wav_list= glob.glob(os.path.join(cwd,okNok, curPre,'*.wav'))
    return wav_list

def data_transfer(w,okNok,curPre):

    mel_list = glob.glob(os.path.join(w,okNok, curPre,'*_mfcc.npy'))
   
    X_mel = np.array([np.load(m) for m in mel_list])
    #assert X_pre.shape == X_cur.shape
    #X = np.stack((X_pre, X_cur),axis= 3)
    np.save('X_'+okNok+curPre+"_mfcc",X_mel)
    print(X_mel.shape)
    
def main(cwd, okNok, curPre):
    '''concate the mfcc of single wav into one npy file '''
    wav_list = get_wavs(cwd, okNok, curPre)
    for w in wav_list:
        wav_to_mfcc(w,okNok, curPre)
    data_transfer(cwd,okNok,curPre)
  

In [5]:
main('./data_collection/', 'ok','previous')
main('./data_collection/', 'nok','previous')
main('./data_collection', 'ok','current')
main('./data_collection/', 'nok','current')

KeyboardInterrupt: 